{[Click here to read this notebook in Google Colab](https://colab.research.google.com/drive/1nmcirlIlf-aZRkt4nqhRrrwCcCL6u0bf)}

<head><link rel = "stylesheet" href = "https://drive.google.com/uc?id=1zYOH-_Mb9jOjRbQmghdhsmZ2g6xAwakk"></head>

<table class = "header"><tr>
    <th align = "left">EPAT Batch 45 | PRM 1, 2020\04\19</th>
    <th align = "right">Written by: Gaston Solari Loudet</th>
</tr></table>

### Portfolio Optimization

The objective of this notebook is to try to simulate the dynamics in a multi-strategy trading account by means of a gambling analogy.

The range of possible results in a casino game grows exponentially both with:
* The amount of wagers "$N_{w}$" in which the gambler <u>sequentially</u> takes part. (betting rounds)
* The amount of games "$N_{g}$" in which the gambler is <u>simultaneously</u> betting.

So does for the trader that trades a <u>sequence</u> of successive trades in <u>simultaneous</u> trading methodologies. We will use a "``list``" for specifying the nature of the different games "$g$" that are being played at the same time. Each element will be a "``pandas.DataFrame``" with "$N_{r}$" rows, one for each possible result "$r$" in a wager. (<u>examples</u>: win, lose, partial win, partial lose, break-even, etc.)

Two columns, "``Rates``" and "``Probs``", will describe the profit/loss rate for each result, and its probability, respectively. Needless to say that for each possible "``Rate``" in any game, there must be one and only one "``Prob``" value between 0 and 1. And they must all add up to zero.

In [72]:
import sympy, numpy, pandas, matplotlib.pyplot, IPython
matplotlib.pyplot.style.use("https://drive.google.com/uc?id=1TawPXELPzNlySaOx3HT67A-wHwRxuWjQ&")
######################################################################################################################
### Each game data is represented as one row in the input dict.                                                    ###
### Dataframe rows correspond to possible outcome events, and columns correspond to stats/variables (Rates/Probs). ###
######################################################################################################################
Games = [ pandas.DataFrame({"Rates": [2.5, -1.0], "Probs": [0.5, 0.5]}),
          pandas.DataFrame({"Rates": [1.5, -1.0], "Probs": [0.5, 0.5]}) ]

### Main class

We will use a "``GameSet``" object that is intended to hold all the relevant data about the gambling games. Besides from the "``list``" of "``.Games``", we will create an array of variables representing possible fractions of our wealth to be bet; one variable "$f_{(g)}$" for each game "$g$". These are created with the "<code>sympy</code>" Python library that allows algebraic manipulation of mathematical "``symbol``" objects. Equations can then be expressed and worked upon in a generic way, before settling numerical values.

Our class holds such in a "``.f_vars``" array, which is going to be created after verifying that the input data (game list) has been adequately expressed (hence the amount of "``assert``" lines).

> <u>**Note**</u>: Most of these elements are not wholly necessary for the solution of the exercise. They are just for the sake of coding practice. I also wanted to try a class programming methodology in which not all the code with the methods and attributes were programmed in a single Jupyter cell, but could be partitioned and continuously updated with new features, taking advantage of the "<code>super</code>" keyword in Python.

In [73]:
class GameSet(): # Basic class structure.

    # Instance constructor.
    def __init__(self, Games):
        self.f_vars = []
        self.Games = []
        # Preprocess data.
        self = self + Games

    # What to show when printing the instance.
    def __repr__(self):
        for g, Game in enumerate(self.Games):
            print(f"\nGame {g + 1}")
            IPython.display.display(Game)
        return ""

    # Allow appending of new games.
    def __add__(self, Games):
        # We only need the list of dataframes.
        if isinstance(Games, GameSet): Games = self.Games
        # Error check
        assert(isinstance(Games, list))
        assert(len(Games) > 0)
        for g, Game in enumerate(Games):
            assert(isinstance(Game, pandas.DataFrame))
            assert("Rates" in Game.columns)
            assert("Probs" in Game.columns)
            # Check that data is numeric.
            assert(Game["Rates"].dtype == "float64")
            assert(Game["Probs"].dtype == "float64")
            # 1 probability value per possible outcome.
            assert(Game["Rates"].shape == Game["Probs"].shape)
            # Build symbolic variables for position sizes.
            self.f_vars.append(sympy.symbols("f_%d" % (g + 1)))
            # Normalize probabilities just in case.
            Game["Probs"] /= sum(Game["Probs"])
        # Keep processed game data.
        return self.Games.extend(Games)

This first instance of the class shows the DataFrame structure when printed, by means of the "``__repr__``" method.

In [74]:
# Create test instance
G = GameSet(Games)   ;   print("GameSet instance dataframes:\n")   ;   print(G)

GameSet instance dataframes:


Game 1


,Rates,Probs
0,2.5,0.5
1,-1.0,0.5



Game 2


,Rates,Probs
0,1.5,0.5
1,-1.0,0.5


#### Symbolic profit rates

When playing a single game "$g$" not simultaneously with any other one, our wealth is multiplied by "$1 + X_{(g;\;r)}$" after each wager. "$X_{(g;\;r)}$" is the <b>random</b> profit/loss rate that represents the result "$r$" that came to be. Let also "$n_{(g;\;r)} $" be the amount of wagers that each result "$r$" took place. The compound profit "$P$" is dictated by:

$$ P = \Pi_{r = 1}^{N_{r}} (1 + X_{(g;\;r)})^{n_{(g;\;r)}} \tag{eq.1} $$

We define the <u>geometric mean profit</u> as the $N_{w}$-th root of the net profit after playing "$N_{w}$" wagers. as "$N_{w}$" approaches infinity, the exponents of the growth factors "$1 + X_{(g;\;r)}$" approach probability values "$p_{(g;\;r)}$":

$$ \bar{P} = \Pi_{r = 1}^{N_{r}} (1 + X_{(g;\;r)})^{p_{(g;\;r)}}$$

When an amount of "$N_{g}$" games are being played in parallel, the growth rates do not longer come the consequence of a single game "$g$" but the sum of all of them. The possible results grow exponentially because they are now the different combinations of the possible outcomes for each single game. Therefore, the probability values in the exponents multiply themselves too:

$$ \bar{P} = \Pi_{r = 1}^{N_{r}} (1 + \Sigma_{g = 1}^{N_{g}} X_{(g;\;r)})^{\Pi_{g = 1}^{N_{g}} p_{(g;\;r)}} $$

It's useful to contemplate entities "$X_{(g;\;r)}$" and "$p_{(g;\;r)}$" as a sort of multidimensional matrix/grid in which:
* Axes represent games "g". Each game is a sort of dimension itself.
* Elements represent "points in those dimensions/axes", which in turn represent combined rates and probabilities.
In a two dimensional case:

<center><img width = "75%" src = "https://drive.google.com/uc?id=1UCG3vP5Vs_saRqwJbjJH1vDrgEzhtFzW"></img></center>

A similar effect is achieved with the "``numpy.meshgrid``" function in the next "``.find_rates``". Attributes called "``.Rates``" and "``.Probs``" will store these grids of growth rates and probability values. Now; the growth "``.Rates``" ("$X_{(g;\;r)}$") depend on:
* The <u>payout ratio</u> "$R_{(g;\;r)}$" (sometimes called "edge") of the possible outcome/result "$r$" at game "$g$".
<br>In trading, this can be interpreted as the stop-profit/stop-loss ratio.
* The <u>bet size</u> "$f_{(g)}$": precisely, the fractional amount of wealth that was trusted in game "$g$".
<br>These are the "``symbol``-ic" variables in "``.f_vars``" that we need to mathematically manipulate.

So: "$X_{(g;\;r)} = R_{(g;\;r)} \cdot f_{g}$". Therefore, the geometric mean profit is finally defined as:

$$ \tag{eq.2} \bar{P} = \Pi_{r = 1}^{N_{r}} (1 + \Sigma_{g = 1}^{N_{g}} R_{(g;\;r)} \cdot f_{(g)})^{\Pi_{g = 1}^{N_{g}} p_{(g;\;r)}} $$

Note that to be able to compute non-vectorial equation 2, we make use of the "``numpy.dot``" function for the summation term and the application of probability exponents. Thus meshgrid attributes "``.Rates``" and "``.Probs``" need their rows to be horizontally stacked with "``numpy.reshape``".

In [75]:
class GameSet(GameSet): # Update class structure.

    # Instance constructor inheritance.
    def __init__(self, Games):
        super().__init__(Games)
        self.find_rates()

    # Generate multivariate mesh of result combinations.
    def find_rates(self):
        Rates = [numpy.array(Game["Rates"]) for Game in self.Games]
        Rates = numpy.array(numpy.meshgrid(*Rates)).T
        Rates = numpy.reshape(Rates, (-1, len(Games)))
        self.Rates = numpy.dot(Rates, self.f_vars)
        # Generate multivariate mesh of probabilities.
        Probs = [numpy.array(game["Probs"]) for game in Games]
        Probs = numpy.array(numpy.meshgrid(*Probs)).T
        Probs = numpy.reshape(Probs, (-1, len(Games)))
        # Each rate prob is product of results from individual games.
        self.Probs = numpy.prod(Probs, axis = 1, keepdims = False)

    # Calculate numeric results with specified bet sizes.
    def calc_rates(self, f_vals):
        # Substitute symbolic variables with input values.
        subs = numpy.array([self.f_vars, f_vals]).T
        # Calculate array of values of the form "(1 + X)".
        Factors = [1 + Rate.subs(subs) for Rate in self.Rates]
        # Return as numpy array.
        return numpy.array(Factors).astype("float")

The method "``calc_rates``" method's output is the numerical value of "$1 + X_{(g;\;r)}$" array elements when a certain value set for "$f_{(g)}$" values is chosen/tested. These "factors" may be further processed as equation 2 shows, to get a geometric mean profit value.

In [76]:
# Renew instance with updated class.
G = GameSet(Games)

print("List of possible outcomes/results:\n")
for case in range(G.Rates.size): print(f"Case {case + 1}  |  Probability: {G.Probs[case]}  |  Growth rate: {G.Rates[case]}")

List of possible outcomes/results:

Case 1  |  Probability: 0.25  |  Growth rate: 2.5*f_1 + 1.5*f_2
Case 2  |  Probability: 0.25  |  Growth rate: 2.5*f_1 - 1.0*f_2
Case 3  |  Probability: 0.25  |  Growth rate: -1.0*f_1 + 1.5*f_2
Case 4  |  Probability: 0.25  |  Growth rate: -1.0*f_1 - 1.0*f_2


#### Profit logarithm

As we are talking about compound profit "$P$", equation 2's nature is purely multiplicative. However, it is easier to manipulate linearly combined variables. So we may apply logarithm to both sides of said equation to seize its linear properties, such as the lowering of probability exponents:

$$ ln\;\bar{P} = \Sigma_{r = 1}^{N_{r}} \; (\Pi_{g = 1}^{N_{g}} p_{(g;\;r)}) \; ln (1 + \Sigma_{g = 1}^{N_{g}} R_{(g;\;r)} \cdot f_{(g)}) \tag{eq.3a}$$

One of its main advantages is that we may be able to find numerous statistical properties of newfound random variable "$ln\;P$" and later extrapolate to the original compound profit $P$. For example, its standard deviation "$\sigma$".

We know that the mean profit logarithm is the expected value of the net profit's logarithm: "$ln\;\bar{P} = E(ln\;P)$". On the other hand, its variance is defined as: "$V(ln\;P) = E(ln^{2}P) - E(ln\;P)^{2} = E(ln^{2}P) - (ln\;\bar{P})^{2}$". Where:
$$ E(ln^{2}P) = \Sigma_{r = 1}^{N_{r}} \; (\Pi_{g = 1}^{N_{g}} p_{(g;\;r)}) \; ln^{2} (1 + \Sigma_{g = 1}^{N_{g}} R_{(g;\;r)} \cdot f_{(g)}) \tag{eq.3b}$$

This value can easily be calculated taking advantage of methods in "``numpy``" such as "``numpy.dot``". Standard deviation can be later calculated as the square root of variance: "$\sigma = \sqrt{V(ln\;P)}$"

In [77]:
class GameSet(GameSet):

    # Instance constructor inheritance and replacement.
    def __init__(self, Games):
        super().__init__(Games)

    # Calculate expected value and standard deviation...
    # ...of geometric profit rate, given bet sizes.
    def calc_stats(self, f_vals):
        # Logarithm of multiplicative growth factors.
        logFactors = numpy.log(self.calc_rates(f_vals))
        E_log = (self.Probs*(logFactors)).sum() # Equation 3a
        E_log_2 = (self.Probs*(logFactors**2)).sum() # Equation 3b
        V_log = E_log_2 - E_log**2 # Variance definition.
        # Mean and standard deviation of profit logarithm.
        mu, sigma = E_log, numpy.sqrt(V_log)
        # Back to compound profit:
        return numpy.exp(mu), numpy.exp(sigma)

Let's give a try of these stats for some arbitrary test values...

In [78]:
# Renew instance with updated class.
G = GameSet(Games)

# Test some combination of bet sizes:
f_test = [0.25, 0.15]
profit_mean, profit_stdv = G.calc_stats(f_test)
print(f"For a set with wealth fraction bets of {f_test}")
print("Geometric mean profit: %.4f" % profit_mean)
print("Standard deviation:    %.4f" % profit_stdv)

For a set with wealth fraction bets of [0.25, 0.15]
Geometric mean profit: 1.1240
Standard deviation:    1.5358


#### Jacobian matrix

Our main target in this exercise is to find the value of "$f_{(g)}$" variables which maximize equation 2. For that, we would need to work with its derivative being equal to zero. An easy way to do this is to use the concept "logarithmic derivative": when a function like "$\bar{P}$" increases  monotonically in a region of values, function "$ln\;\bar{P}$" does so too. Then, minima/maxima settle in the exact same points.

When we differentiate equation 3a: "$d\;ln\;\bar{P} = (1/\bar{P}) \cdot d\bar{P}$". As we are forcing "$d\bar{P} = 0$", then "$d\;ln\;\bar{P} = 0$" as well.<br>On the other hand, each term in the summation is also differentiated:

$$ \frac{\partial \; ln\;\bar{P}}{\partial f_{g}} = \Sigma_{r = 1}^{N_{r}} \; \frac{\Pi_{g = 1}^{N_{g}} \; p_{(g;\;r)}}{1 + \Sigma_{g = 1}^{N_{g}} \; R_{(g;\;r)} \cdot f_{(g)}} \; R_{(g;\;r)} = 0 \tag{eq.4} $$

Note that the factors "$1 + X_{(r)}$" in the denominators of equation 4 take the form of a linear combination of the bet wealth fractions "$f_{(g)}$":

$$ 1 + X_{(r)}(f_{(1)}; \;f_{(2)}; \;\cdots; \;f_{(g)}; \;\cdots; \;f_{(N_{g})}) = 1 + R_{(1;\;r)} \cdot f_{(1)} + R_{(2;\;r)} \cdot f_{(2)} + \cdots + R_{(g;\;r)} \cdot f_{(g)} + \cdots + R_{(N_{g};\;r)} \cdot f_{(N_{g})} $$

When differentiating them by "$f_{(g)}$", every term is cancelled except for the one holding such variable: "$ d(1 + X_{(r)})/df_{(g)} = R_{(g;\;r)} $". So:

$$ X_{(r)}(0; \;0; \;\cdots; \;1; \;\cdots; \;0) = 0 + 0 + \cdots + R_{(g;\;r)} \cdot 1 + \cdots + 0 = R_{(g;\;r)} = d(1 + X_{(r)})/df_{(g)} $$

As in "``.Rates``" we've already got the summations, it's OK to replace "$f_{(g)}$" values by zeros and one accordingly to find the derivative of the denominator for each case. This is the role of "``.diff_rates``" method: to assemble a Jacobian matrix that could be later dot-multiplied together with "``.Probs/(1 + .Rates)``" so as to get a system of equations.

In [79]:
class GameSet(GameSet):

    # Instance constructor inheritance and replacement.
    def __init__(self, Games):
        super().__init__(Games)
        self.diff_rates()

    # Symbolic Jacobian matrix computation.
    def diff_rates(self):
        size = (len(self.f_vars), self.Rates.size)
        self.dRates = numpy.random.random(size = size)
        for i in range(len(self.f_vars)):
            # (f1; ...; fi; ...; fn) = (0; ...; 1; ...; 0)
            f_subs = numpy.identity(n = len(self.f_vars))[i, :]
            for j in range(self.Rates.size):
            # Notice that: "R = R1 x f1 + ... + Ri x fi + ... + Rn x fn"
            # So: "dR/dfi = Ri = R1 x 0 + ... + Ri x 1 + ... + Rn x 0"
                subs = numpy.array([self.f_vars, f_subs]).T
                self.dRates[i, j] = self.Rates[j].subs(subs)

The next code shows how "``.diff_rates``" returns a numerical matrix.

In [80]:
# Renew instance with updated class.
G = GameSet(Games)

print("List of derivatives for each case:\n")
for case in range(G.Rates.size): 
    string = f"Case {case + 1}"
    for var in range(len(G.f_vars)):
        f_var = str(G.f_vars[var])
        dRate = str(G.dRates[var, case])
        string += f" | dR/d{f_var}: {dRate}"
    print(string)

List of derivatives for each case:

Case 1 | dR/df_1: 2.5 | dR/df_2: 1.5
Case 2 | dR/df_1: 2.5 | dR/df_2: -1.0
Case 3 | dR/df_1: -1.0 | dR/df_2: 1.5
Case 4 | dR/df_1: -1.0 | dR/df_2: -1.0


#### Optimization

Note that there will be as many equation 4 instances as games we are playing. As the amount of "$f_{(g)}$" variables we are looking for in our solution, is exactly the same, we are in presence of a <u>compatible system of non-linear equations</u>. Now: one could force Python to "``sympy.solve``" with numerical methods. However, for a large number of games, this could take some real time. Therefore, we include the possibility of using a "``Taylor``" polynomial approach to the algorithm, knowing that: "$1/(1 + x) \approx \Sigma_{t = 0}^{T}\;(-x)^{t}$".

Despite massively speeding up the "``sympy.solve``" process, this method has two important disadvantages: its error margin and the multiplicity of solution sets. Care must be taken of the latter, eliminating not feasible solutions knowing that:
* They must be real (no imaginary part).
* Their sum must be smaller than 1: "$\Sigma_{g = 1}^{N_{g}} \; f_{(g)} \leq 1$".

In [81]:
class GameSet(GameSet):

    # Instance constructor inheritance and replacement.
    def __init__(self, Games):
        super().__init__(Games)
    
    # Find optimal set of bet sizes.
    def optimize(self, taylor = None):
        # Use polynomial approximation.
        if isinstance(taylor, int):
            assert(taylor > 0)
            Taylor = 1
            # "1/(1 + x) ~ 1 - x + x^2 - ..."
            for deg in range(1, taylor + 1):
                Taylor += (-self.Rates)**deg
            dX_df = self.dRates*Taylor
        else: # Evaluate real solution.
            dX_df = self.dRates/(1 + self.Rates)
        # Derivative log geometric mean of profit.
        dG_df = numpy.dot(dX_df, self.Probs)
        # Solve system of equations.
        f_vals = sympy.solve(dG_df, self.f_vars)
        # Process solution.
        if isinstance(f_vals, dict):
            # Single solution usually presented as dict.
            f_vals = list(f_vals.values())
        f_vals = numpy.array(f_vals)
        # Multiple solutions usually are complex.
        if f_vals.ndim > 1:
            for i in range(len(f_vals)):
                for j in range(len(f_vals[i])):
                    # Delete complex solutions.
                    f_vals[i][j] = sympy.re(f_vals[i][j])
        # Keep the ones representing a percentage.
        f_vals = f_vals[(0 <= f_vals)*(f_vals < 1)]
        return f_vals.astype("float")

The next code gets the optimal values of "$f_{(g)}$" with "``.optimize``". Geometric mean profit is calculated as shown.

In [82]:
# Renew instance with updated class.
G = GameSet(Games)

f_vals = G.optimize()
print(f"Optimal fraction configuration: {f_vals}")
p_max = (G.calc_rates(f_vals = f_vals)**G.Probs).prod()
print(f"Geometric mean compound profit: {p_max}")

Optimal fraction configuration: [0.29292863 0.14242735]
Geometric mean compound profit: 1.126160682927342


#### Meshplot
It's useful to have a function to visualize the variation of geometric mean profit "$\bar{G}$" with different bet sizes "$f_{(g)}$". This is only possible when the amount of games allows it: "$N_{g} \leq 2$". A contour plot can be created for the bidimensional case. We may also visually locate the calculated optimal point in this plot. Note that due to the constraints imposed over the bet sizes, there are certain regions in the "``meshgrid``" which lose sense (for example, the ones which represent wealths larger than 100%, what we actually have).

In those cases in which we want to focus on a smaller region of the plane that's nearer to the origin, we can use the "``limit``" input.

In [83]:
class GameSet(GameSet):

    # Instance constructor inheritance and replacement.
    def __init__(self, Games):
        super().__init__(Games)

    # Build mesh matrix for different bet sizes.
    def meshgrid(self, points = 5, limit = [1, 1]):
        # 2D array plotting would only work for 2 games.
        assert(len(self.f_vars) == 2)
        # Makes no sense to see what happens beyond 100% wealth.
        assert((limit[0] <= 1) and (limit[1] <= 1))
        # Generate axis with points.
        range_X = numpy.linspace(0, limit[0], points + 1)
        range_Y = numpy.linspace(0, limit[1], points + 1)
        # "X" for "f_1" and "Y" for "f_2"
        X, Y = numpy.meshgrid(range_X, range_Y)
        # "Zm" for mean, "Zs" for standard dev.
        Zm = numpy.ones_like(X)*numpy.nan
        Zs = numpy.ones_like(X)*numpy.nan
        # "x" and "y" are each point's indexes.
        for x in range(points):
            # As "f_1 + f_2 <= 1"; "x + y <= points"
            for y in range(points):
                f_1, f_2 = X[x, y], Y[x, y]
                # Omit the points beyond 100% wealth.
                if (f_1 + f_2 >= 1): pass
                else:
                    Zm[x, y], Zs[x, y] = self.calc_stats([f_1, f_2])
        return X, Y, numpy.matrix(Zm, dtype = "float"), \
                     numpy.matrix(Zs, dtype = "float"),

The more "``points``" are used for the "``meshgrid``", the more accurate is the plot. Also "same profit" curves become clearer.

In [84]:
% # Delete "%" to run this cell. It will work only if the system is 2D: 2 games.
G = GameSet(Games)    ;    X, Y, Zm, Zs = G.meshgrid(points = 50, limit = [1, 1])
##################################################################################
### Following code is just for plot configuration. Not very important to read. ###
##################################################################################
def figure_1(zmax):
    Figure = matplotlib.pyplot.figure(figsize = (18, 4))
    Figure.add_axes([0.1, 0.1, 0.6, 0.8])
    Figure.axes[0].set_xlabel(fontweight = "bold", xlabel = "Game 1")
    Figure.axes[0].set_ylabel(fontweight = "bold", ylabel = "Game 2")
    Figure.axes[0].set_xlim(xmin = 0, xmax = 1)  ;  Figure.axes[0].set_xticks(numpy.arange(0, 1, 0.05))
    Figure.axes[0].set_ylim(ymin = 0, ymax = 1)  ;  Figure.axes[0].set_yticks(numpy.arange(0, 1, 0.05))
    # Colorbar for threshold identification.
    Figure.add_axes([0.7, 0.1, 0.1, 0.8])
    norm = matplotlib.colors.Normalize(vmin = 0, vmax = zmax*1.1)
    colors = matplotlib.cm.ScalarMappable(norm = norm)
    colorbar = matplotlib.pyplot.colorbar(mappable = colors, cax = Figure.axes[1])
    colorbar.set_ticks(numpy.arange(0, zmax*1.1, zmax/11))
    Figure.axes[1].set_xlabel(fontsize = 12, xlabel = "Geometric\nmean profit")
    Figure.axes[1].xaxis.set_label_coords(0.5, -0.025)
    matplotlib.pyplot.pause(1e-13)
    return Figure
######################################################################## Let's try.
Figure_0a = figure_1(zmax = round(numpy.nanmax(Zm), 1))
Figure_0a.axes[0].contour(X, Y, Zm, levels = 2*X.shape[0])
# Localize optimal values formerly calculated.
Figure_0a.axes[0].plot(*f_vals, marker = "X", markeredgecolor = "white")
Figure_0a.axes[0].text(*(f_vals + 0.01), s = "Optimal profit = %.4f" % p_max,
                        fontweight = "bold", fontsize = 12, color = "white");

UsageError: Line magic function `%` not found.


<center><img width = "100%" src = "https://drive.google.com/uc?id=1ZNaRA6sQiPZ49dLBs45_7ABw1uQMnqSx"></img></center>

Let's try and plot the same mesh but for the logarithmic standard deviations "$\sigma$". Notice that its direction of growth is opposite to the geometric mean profit.

In [91]:
% # Delete this line to run this cell.
Figure_0b = figure_1(zmax = 1.1);
Figure_0b.axes[0].contour(X, Y, numpy.log(numpy.multiply(Zs, Zs < 10)), levels = 2*X.shape[0])
# Localize optimal values formerly calculated.
Figure_0b.axes[1].set_xlabel(fontsize = 12, xlabel = "Standard dev.\n of 'ln(G)'")
Figure_0b.axes[0].plot(*f_vals, marker = "X", markeredgecolor = "white")
Figure_0b.axes[0].text(*(f_vals + 0.01), s = "Optimal profit = %.4f" % p_max,
                        fontweight = "bold", fontsize = 12, color = "white");

UsageError: Line magic function `%` not found.


<center><img width = "100%" src = "https://drive.google.com/uc?id=1ANJvV7fYveQgUE_OyZT8Ms6pGDOv8afn"></img></center>

#### Risk of ruin

Optimizing the investment distribution does not guarantee the positivity of the results. Though sometimes theoretically unlikely, continuous losses may occur and drain our accounts. But - **which are the chances?**

Up to know we have talked about the net profit in statistical terms, taking into account only the sample space of results in a non-sequential way. But when the "time" variable becomes considered, the order in which "$1 + X_{(g;\;r)}$" factors appear, matters. Then, we shall develop a "``.ruin_cases``" method in which, for a certain sequence of "``wager``" rounds, the "``product``" between the sequence of "``factors``" of the form "$1 + X_{(g;\;r)}$" is calculated. If the product goes lower than a certain "``threshold``" , that result plus also all of the future possible scenarios that are born from it, imply a "ruin" condition.

The following image gives an example of how may the algorithm work. It uses some principles of dynamic programming:

<center><img width = "50%" src = "https://drive.google.com/uc?id=1B9YyB-5xDU9Tc7Z62tSpq2PDQH4z_MiD" width = "600"></center>

In [86]:
class GameSet(GameSet):

    # Instance constructor inheritance and replacement.
    def __init__(self, Games):
        super().__init__(Games)

    # Finds amount of "ruining" cases in a possibility tree by recursion.
    @staticmethod
    def ruin_cases(factors, wager, threshold = 1, counter = 0, product = 1):
        # "threshold" represents the minimum wealth needed to continue playing.
        # If ongoing product is below threshold, don't continue on that branch.
        if (product < threshold):
            # Just add to the counter all derived results at wagers ahead.
            return counter + len(factors)**wager
        # If no more wagers ahead, I'm at the end of a branch. Go back.
        if (wager == 0): return counter
        # Select next branch.
        for factor in factors:
            counter = GameSet.ruin_cases(factors = factors, wager = wager - 1,
                threshold = threshold, counter = counter, product = product*factor)
        # If no more branches besides, I'm at the end of a wager. Go back.
        return counter
            
    # Finds risk-of-ruin probability curve after a certain number of wagers.
    @staticmethod
    def ruin_risk(factors, wagers, threshold = 1):  
        ROR = []
        factors = list(factors)
        for wager in range(1, wagers + 1):
            N_ROR = GameSet.ruin_cases(factors = factors,
                wager = wager, threshold = threshold)
            N_TOT = len(factors)**wager
            ROR.append(N_ROR/N_TOT)
        return numpy.array(ROR)

The risk of ruin "$p_{ROR}$" calculated by the method "``.risk_of_ruin``" is by definition, a probability value that is computed as the fraction of possible cases for a wager value "$w$" whose product has gone below the threshold, over all possible results. We already know the amount of possible results as each one of the "$N_{ROR}$" ``factors`` represent a possible result, and these conform an exponentially growing "possibility tree" as successive wagers "$w$" increase. So then:

$$ p_{ROR} = \frac{N_{ROR}(w)}{N_{r}^{w}} \tag{eq.4}$$

In [87]:
% # Delete "%" to run this cell. It will work only if the system is 2D: 2 games.
G = GameSet(Games)   ;   wagers = 12
##############################################################################
##### Following code is just for plot config. Not very important to read #####
##############################################################################
def figure_2(wagers):
    Figure = matplotlib.pyplot.figure(figsize = (18, 4))
    Figure.add_axes([0.1, 0.1, 0.6, 0.8])   ;  rr = "Risk of ruin $\Rightarrow$"
    Figure.axes[0].set_xlabel(fontweight = "bold", xlabel = "Number of wagers")
    Figure.axes[0].set_ylabel(fontweight = "bold", ylabel = rr, rotation = 0, ha = "right")
    Figure.axes[0].set_xlim(xmin = 1, xmax = wagers) ; Figure.axes[0].set_xticks(numpy.arange(1, wagers + 1))
    Figure.axes[0].set_ylim(ymin = 0, ymax = 1)      ; Figure.axes[0].set_yticklabels(labels = []);
    Figure.axes[0].yaxis.set_label_coords(0.99, 0.925); Figure.axes[0].yaxis.tick_right(); 
    # Colorbar for threshold identification.
    Figure.add_axes([0.7, 0.1, 0.1, 0.8])
    norm = matplotlib.colors.Normalize() ; cmap = matplotlib.cm.jet
    colors = matplotlib.cm.ScalarMappable(norm = norm, cmap = cmap)
    colorbar = matplotlib.pyplot.colorbar(mappable = colors, cax = Figure.axes[1])
    colorbar.set_ticks(numpy.arange(0, 1, 0.1))
    Figure.axes[1].set_xlabel(fontweight = "bold", xlabel = "Wealth\nthresholds")
    matplotlib.pyplot.pause(1e-13)
    return Figure
######################################################### Risk of ruin curves.
Figure_0d = figure_2(wagers = wagers);
factors = G.calc_rates(f_vals = f_vals);
for threshold in (numpy.arange(1, 11)*0.1):
    RR = GameSet.ruin_risk(factors, wagers = wagers, threshold = threshold)
    Figure_0d.axes[0].plot(range(1, wagers + 1), RR, color = matplotlib.cm.jet(threshold), linewidth = 2);

UsageError: Line magic function `%` not found.


<center><img width = "100%" src = "https://drive.google.com/uc?id=15BNFM2loR2k5oHbHPgMgB29RRn91MDOK"></img></center>

It may seem interesting to plot above some risk-of-ruin curves for different threshold values. One may appreciate that all of them approach asymptotically towards a certain max value. This "max value" concept could be taken as benchmark for future trading strategy evaluations.

<hr><hr><hr>

#### Exercise 1

> There are 2 coin toss games that are happening in parallel, say Game 1 and Game 2. Both the coins are unbiased: there are equal probabilities of getting both head and tail.<br><br>However, the payouts from the games are different:

> 1. "Win - loss" ratio is **2.5 : 1**… ("$R_{(1)} = 2.5$")
  2. "Win - loss" ratio is **1.5 : 1**… ("$R_{(2)} = 1.5$")

> <u>**Rules**</u>:
> * The 2 games are played simultaneously.
  * You can put a portion of your total wealth to bet on each game.
  * After each round of betting, you can pool your gains/losses from the results of both games…
  * …then again distribute a portion of your total wealth into them, and continue the process.

> <u>**Question**</u>: What percentage of your total wealth should you bet in Game 1 ("$f_{(1)}$") and in Game 2 ("$f_{(2)}$")? Should you adopt any other methodologies?

Just in case, let's create a "``GameSet``" instance for the games mentioned in the exercise. Let's also calculate the optimal wealth fractions "``f_opt``" for betting.

In [88]:
Games = [ pandas.DataFrame({"Rates": [2.5, -1.0], "Probs": [0.5,  0.5]}),
          pandas.DataFrame({"Rates": [1.5, -1.0], "Probs": [0.5,  0.5]}) ]

G = GameSet(Games)

f_opt = G.optimize()
print("Optimal wealth fractions: %s" % f_opt)
print("Total wealth fraction gambled: %.4f" % f_opt.sum())
m, s = G.calc_stats(f_opt)
print("Highest geometric mean profit: %.4f ... log: %.4f" % (m, numpy.log(m)))
print("Standard deviation cmp profit: %.4f ... log: %.4f" % (s, numpy.log(s)))

Optimal wealth fractions: [0.29292863 0.14242735]
Total wealth fraction gambled: 0.4354
Highest geometric mean profit: 1.1262 ... log: 0.1188
Standard deviation cmp profit: 1.6218 ... log: 0.4835


There are "$N_{r} = 4$" possible results for each wager. We can see that eventhough the geometric mean profit "$\bar{G}$" is positive. However, its standard deviation is pretty high. In fact, its logarithmic value is over "$ln\;\bar{G}$" itself, so results can be pretty volatile.

A good approach would be to apply a fraction of these wealth portions that are being bet: in both figures below, which were plotted with "``.meshplot``" method above, if connecting the origin ("$f_{(1)} = f_{(2)} = 0$") which has (obviously) zero standard deviation with the optimized point, one could choose a point in the midline to reduce the volatility while sacrificing some profit rate.

<table><tr><th><img src = "https://drive.google.com/uc?id=1-3uhF04tu4G9l0xwU36QtkcMEirXLEyr" width = "100%"></th>
           <th><img src = "https://drive.google.com/uc?id=1jU6GlhGvaabFQ7Jc_C2_VViTmDGycVBC" width = "100%"></th></tr></table>

We can take the advantage of the "``limit``" input in "``meshgrid``" for this.  Using the optimals as input will keep the "$f_{(2)}/f_{(1)}$" ratio in the elements of the diagonals in the output matrices.

In [89]:
% # Delete this line to run this cell. It generates a dense mesh of 100 points per axis.
X, Y, Zm, Zs = G.meshgrid(points = 100, limit = f_opt)
X, Y, Zm, Zs = numpy.diag(X), numpy.diag(Y), numpy.diag(Zm), numpy.diag(Zs)
log_mean = numpy.nan_to_num(numpy.log(Zm), 0); log_mean[-1] = numpy.log(m)
log_stdv = numpy.nan_to_num(numpy.log(Zs), 0); log_stdv[-1] = numpy.log(s)
##############################################################################
Figure = matplotlib.pyplot.figure(figsize = (18, 4))
Figure.add_axes([0.1, 0.1, 0.8, 0.8])
Figure.axes[0].set_xlabel(fontweight = "bold", fontsize = 12, xlabel = "Fraction of optimal pair")
Figure.axes[0].set_ylabel(fontweight = "bold", fontsize = 12, ylabel = "Log profit stats")
Figure.axes[0].set_xlim(xmin = 0, xmax = 1)
Figure.axes[0].set_ylim(ymin = 0, ymax = log_stdv.max())
##############################################################################
Figure.axes[0].plot((X + Y)/f_opt.sum(), log_mean, color = "lime", label = "Mean")
Figure.axes[0].plot((X + Y)/f_opt.sum(), log_stdv, color = "red", label = "Standard Deviation")
Figure.axes[0].plot((X + Y)/f_opt.sum(), log_mean/log_stdv, color = "yellow", label = "Sharpe ratio")
Figure.axes[0].legend(fontsize = 12); matplotlib.pyplot.pause(1e-13)

UsageError: Line magic function `%` not found.


<center><img width = "100%" src = "https://drive.google.com/uc?id=1vA6Nxi5jbfh-tQ3aXcJlXNmUO5zSXPqv"></img></center>

It's clear that the standard deviation grows much faster than the mean. That's why Sharpe decays so aggresively. Therefore, volatility is unavoidable. The only solution is to look for some point "``f_less``" which having "$ln\;\sigma \leq 2.25 \cdot ln\;\bar{G}$", is nearest to 1. That is: "$Sharpe = 4/9$". Then, we shall test it for risk of ruin curves.

In [90]:
% # Delete this line to run this cell. It draws risk of ruin curves for specified sharpe ratio.
f_less = ((X + Y)/f_opt.sum())[log_stdv <= 2.25*log_mean].max()
f_frac = numpy.array(f_less)*numpy.array(f_opt)
print("New fractional bet size from optimal: %s: " % f_frac)
# Risk of ruin curves for different threshold values.
Figure_1d = figure_2(wagers = 12)
factors = G.calc_rates(f_vals = f_frac)
for threshold in (numpy.arange(1, 11)*0.1):
    RR = GameSet.ruin_risk(factors, wagers = 12, threshold = threshold)
    Figure_1d.axes[0].plot(range(1, 12 + 1), RR, color = matplotlib.cm.jet(threshold), linewidth = 2);

UsageError: Line magic function `%` not found.


<center><img width = "100%" src = "https://drive.google.com/uc?id=1E0BK0qv2Aupn8GrxRxEkFwGVKYXQ3SpH"></img></center>

Upper line in dark red represents the probability of our wealth to decrease from initial amount (threshold is 1). Its asymptote settled in about 55%, and the ones below settled much lower. So eventhough the standard deviation is still high, this approach is relatively conservative and has about 45% chance of definitively profitting at long term.

So: <u>the chosen combination is around "3.5%" for game 1 and "1.7%" for game 2</u>.

<hr><hr><hr>

#### Exercise 2

> There are now "infinite" coin toss games occurring in parallel; say game "$g$" with "$g = 1, 2, \ldots$". All of them have the same “Win – loss” ratio of 2 : 1 ("$R = 2$"). Rules are exactly the same as before.

> <u>**Questions**</u>:
> * While the geometric rate of return for a particular period increases on playing more than one game in parallel... what value does it asymptotically lead to?
  * Are there any drawdowns of playing multiple games in parallel? Thus, what will your strategy be?
  * Kindly state all scenarios and how you would play the game according to each.

> Say scenarios like...
> * ...incorporating risk of ruin,
  * ...ability to earn interest on money not put into the game,
  * ...biased coin (heads more probable than tails or otherwise),
  * ...probability of runs (probability of win after win, or otherwise)

Let's take care of the most general case in which we've got...
* An infinite amount of identical games "$N_{g} \rightarrow \infty$".
* Total gambled wealth is "$f_{0}$", equally divided between games.
* Everyone presents a win-loss payout ratio "$R:1$".
* Probabilities are biased: "$p_R : p_1$" may be different than 1:1.
* Zero-risk growth rate "$R_{0}$" on ungambled money ("$1 - f_{0}$").

For a <u>single game</u>, just 2 factors define our random log profit "$ln\;P$". Let's call the difference between the random log profit and new log zero risk growth as "$ ln\;Q = ln\;P - ln(1 + (1 - f_{0})R_{0}) $". As described by equation 3a and with all data gathered above, the mean/"expected value" of this random variable "$ln\;Q$" would be:

$$ E(ln\;{Q}) = p_{R} \; ln(1 + R \cdot f_{0}) + p_{1} \; ln(1 - f_{0}) = \mu $$

Let's then apply equation 3b to find its variance:

$$ E(ln^{2}{Q}) = p_{R} \; ln^{2}(1 + R \cdot f_{0}) + p_{1} \; ln^{2}(1 - f_{0}) $$

As by the definition of standard deviation:

$$ V(ln\;Q) = E(ln^{2}{Q}) - E(ln\;{Q})^{2} = \sigma^{2}$$

After some term expansion, we get:

$$ \sqrt{V(ln\;Q)} = \sqrt{p_{R} \; p_{1}} \; [\; ln(1 + R \cdot f_{0}) + ln(1 - f_{0}) \;] = \sigma $$

So the dynamics of a single game with two single outcomes, follow a Bernoulli distribution with mean "$\mu$" and standard deviation "$\sigma$". When extended to various games, "$ln\;Q$" becomes a sum of independent terms for each game, with the format of equation 3a. The probabilistic process tends towards a Binomial distribution because each term represents the combination of individual results.

<center><img width = "75%" src = "https://drive.google.com/uc?id=1UCG3vP5Vs_saRqwJbjJH1vDrgEzhtFzW"></img></center>

<ul>So then equation 3a turns into:
$$ ln\;\bar{Q} = \Sigma_{g = 1}^{N_{g}} \; {N_{g}\choose{g}} \; p_{R}^{g}
\; p_{1}^{N_{g} - g} \; ln\;(1 + [R(N_{g} - g) - g]\;\frac{f_{0}}{N_{g}}) $$

When the number of games is very large ("$N_{g} \rightarrow \infty$"), the Binomial distribution becomes a Gaussian Normal distribution.<br>The issue with this fact is that theoretically, the original standard deviation becomes multiplied by the amount of games "$N_{g}$":

$$ lim_{N_{g} \rightarrow \infty} \; \sigma = N_{g} \sqrt{p_{R} \; p_{1}} \; [\; ln(1 + R \cdot f_{0}) + ln(1 - f_{0}) \;] $$

So therefore it's not a good idea to play an increasingly large amount of games at the same time: <u>**the log profit becomes more volatile**</u>.

Also there are other facts to take into consideration:
* The impossibility of controlling so many games at the same time.
* The indivisibility of wealth beyond a certain discrete fraction.